In [ ]:
import datetime
import glob
import numpy as np
import xarray as xr
import rasterio
import rioxarray
import pandas as pd
from matplotlib import pyplot
from rasterio.warp import Resampling

In [ ]:
file_list = glob.glob('data/processed/VNP16_ET_and_PET/VNP16_ET_mm_8day-1_*.tiff')
file_list.sort()

rasters_et = []
rasters_pet = []
for filename in file_list:
    date_str = filename.split('/')[-1].split('_')[-1].replace('.tiff', '')
    date = datetime.datetime.strptime(date_str, '%Y%m%d')
    et0 = rioxarray.open_rasterio(filename)\
        .rename(band = 'time')\
        .assign_coords(time = [date])
    rasters_et.append(et0)
    # Do the same for PET, note the use of replace()
    pet0 = rioxarray.open_rasterio(filename.replace('ET_mm_8day-1', 'PET_mm_8day-1'))\
        .rename(band = 'time')\
        .assign_coords(time = [date])
    rasters_pet.append(pet0)

# Convert from [mm 8day-1] to [mm day-1]
ds_et0 = xr.concat(rasters_et, dim = 'time').to_dataset(name = 'ET') / 8.0
ds_pet0 = xr.concat(rasters_pet, dim = 'time').to_dataset(name = 'PET') / 8.0

ds_et = xr.merge([ds_et0, ds_pet0])

```
# TODO Resampling to daily time steps; we have to a

ds_et_last = ds_et.isel(time = -1).assign_coords(time = pd.Timestamp('2023-12-31'))
ds_et_daily = xr.concat([ds_et, ds_et_last], dim = 'time')\
    .resample(time = '1D', skipna = False).ffill()
ds_et_daily
```

## Computing the initial soil water content

In [ ]:
# TODO Now loading precipitation
example = rioxarray.open_rasterio('data/processed/VNP16_ET_and_PET/VNP16_ET_mm_8day-1_20230101.tiff')

# Resample the precip data to match our ET data
ds_precip = xr.open_dataset('data/processed/IMERG_precip_mm_day-1_for_study_area.nc4')\
    .rio.write_crs(example.rio.crs)
ds_precip_500m = ds_precip.rio.reproject_match(example, resampling = Resampling.bilinear)

In [ ]:
precip_total = ds_precip_500m.sum('time').precipitation

In [ ]:
swc = precip_total - ds_et.ET.sum('time')

In [ ]:
swc.plot()